# Notebook Recons 
It is a notebook which shows examples with recons:
- how you can aggregate recon events and build a table like 'Recon Folder -> Recon Rule -> Recon Status -> Number recon events'
- how you can aggregate recon events by intervals.

In [1]:
from pprint import pprint
import pandas as pd
from IPython.core.display import display, HTML
from datetime import datetime, timedelta
from th2_data_services.data_source import DataSource
from th2_data_services.data import Data
from th2_data_services.events_tree import EventsTree
import th2_data_services_utils.utils as Utils
from pandas import DataFrame

# This settings for increase display jupyter notebook and dataframe table.
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.max_rows = 1500
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

![alt text](super_type.png "123")

In [2]:
# For understand which event type on based name we get from stream.
def get_super_type(record, tree):
    name = record.get("eventName")
    parent_id = record.get("parentEventId")
    super_type = record.get("eventType")
    if super_type == "":
        if "Recon" in name:
            super_type = "Recon Folder"
        else:
            if not parent_id:
                super_type = "Test Run"
            else:
                parent_event = tree.get(parent_id)
                if parent_event:
                    parent_super_type = get_super_type(parent_event, tree)
                    if parent_super_type == "Test Run":
                        super_type = "Test Case"
                    elif parent_super_type == "Recon Folder":
                        super_type = "Recon Rule"
                    elif parent_super_type == "Recon Rule":
                        super_type = "Recon Status"
                    elif parent_super_type == "Recon Status":
                        super_type = "Recon Event"

    return super_type

# Base extract (transform function)
# record is required arguments.
def extract_basic(record):
    new_object = {}
    start_time = datetime.fromtimestamp(record.get("startTimestamp", {}).get("epochSecond", 0))
    start_time += timedelta(microseconds=record.get("startTimestamp", {}).get("nano", 0))
    end_time = datetime.fromtimestamp(record.get("endTimestamp", {}).get("epochSecond", 0))
    end_time += timedelta(microseconds=record.get("endTimestamp", {}).get("nano", 0))
    new_object.update(
        {
            "super_type": get_super_type(record, tree),
            "start_time": start_time,
            "end_time": end_time,
            "status": "SUCCESSFUL" if record.get("successful") else "FAILED",
            "eventName": record.get("eventName"),
            "parentEventId": record.get("parentEventId"),
            "body": record.get("body"),
            "messages_id": record.get("attachedMessageIds")
        }
    )
    return new_object

## Create Data Source object
The DataSource object lets you retrieve data in the easiest way.

NOTE: You can change the URL via eponymous property of this object.

In [3]:
START_TIME = datetime(year=2021, month=6, day=20, hour=13, minute=44, second=41, microsecond=692724)
END_TIME = datetime(year=2021, month=6, day=20, hour=13, minute=45, second=49, microsecond=28579)

DEMO_HOST = "10.64.66.66"  # th2-kube-demo  Host port where rpt-data-provider is located.
DEMO_PORT = "30999"  # Node port of rpt-data-provider.
data_source = DataSource(F"http://{DEMO_HOST}:{DEMO_PORT}")

events: Data = data_source.get_events_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    metadataOnly=False,
    attachedMessages=True,
)

## This example demonstrates events retrieving


In [4]:
# We build events tree for further assistance.
events_tree = EventsTree(events)
tree = events_tree.events

# Here we get events which doesn't exist in data source interval.
events_tree.recover_unknown_events(data_source)

## [1] Summarize table
The table summarizes the work of the check2-recon th2 component.  
Shows the names of the rules, statuses, and how many there were.

In [5]:
def transform_output(record):
    recon_status = tree.get(record.get("parentEventId"))
    recon_rule = tree.get(recon_status.get("parentEventId"))
    recon_folder = tree.get(recon_rule.get("parentEventId"))
    new_obj = {
        "Recon Folder": recon_folder.get("eventName"),
        "Recon Rule" : recon_rule.get("eventName"),
        "Recon Status": recon_status.get("eventName"),
        "Number of Events": 1,
        "Start Time": record.get("start_time"),
        "End Time": record.get("end_time")
    }
    return new_obj

data: Data = events\
            .map(extract_basic)\
            .filter(lambda record: record.get("super_type") == "Recon Event")\
            .map(transform_output)

In [6]:
def transform_output(record):
    recon_status = tree.get(record.get("parentEventId"))
    recon_rule = tree.get(recon_status.get("parentEventId"))
    recon_folder = tree.get(recon_rule.get("parentEventId"))
    new_obj = {
        "Recon Folder": recon_folder.get("eventName"),
        "Recon Rule" : recon_rule.get("eventName"),
        "Recon Status": recon_status.get("eventName"),
        "Number of Events": 1,
        "Start Time": record.get("start_time"),
        "End Time": record.get("end_time")
    }
    return new_obj

data: Data = events\
            .map(extract_basic)\
            .filter(lambda record: record.get("super_type") == "Recon Event")\
            .map(transform_output)
    
# Functions from pandas.
df = DataFrame(data).groupby(['Recon Folder', "Recon Rule", 'Recon Status']).agg(
    {"Number of Events": "sum", "Start Time": "min", "End Time": "max"})

df = Utils.append_total_rows(df, {"Number of Events": "sum", "Start Time": "min", "End Time": "max"})
df["duration"] = df["End Time"] - df["Start Time"]
df

Number of Events              Start Time                End Time               duration
Recon Folder      Recon Rule                                         Recon Status                                                                                           
Recon: Demo_Recon Rule: ['demo-conn1', 'demo-conn2'] vs ['demo-log'] No match                    16.0 2021-06-20 13:55:27.696 2021-06-20 13:59:48.824 0 days 00:04:21.128000
                                                                     total                       16.0 2021-06-20 13:55:27.696 2021-06-20 13:59:48.824 0 days 00:04:21.128000
                  Rule: ['demo-conn1'] vs ['demo-conn2']             Matched failed               1.0 2021-06-20 13:55:24.826 2021-06-20 13:55:24.826        0 days 00:00:00
                                                                     Matched passed               9.0 2021-06-20 13:54:49.498 2021-06-20 14:01:56.822 0 days 00:07:07.324000
                                                                     total                       10.0 2021-06-20 13:54:49.498 2021-06-20 14:01:56.822 0 days 00:07:07.324000

## [2] Growth chart for each of the event statuses



In [10]:
def transform_output(record):
    recon_status = tree.get(record.get("parentEventId"))
    recon_rule = tree.get(recon_status.get("parentEventId"))
    recon_folder = tree.get(recon_rule.get("parentEventId"))
    new_obj = {
        "time": record.get("start_time"),
        "name": f"{recon_folder.get('eventName')} -> {recon_rule.get('eventName')} -> {recon_status.get('eventName')}",
        "status": record.get("status")
    }
    return new_obj

data = events\
        .map(extract_basic) \
        .filter(lambda record: record.get("super_type") == "Recon Event") \
        .map(transform_output)

df = Utils.aggregate_groups_by_intervals(data, "time", "name", intervals="10s", pivot="name")

Utils.create_tick_diagram(df)  # The plot may not be shown if you have not restarted the notebook.

## What do we want to show here?
Work with messages

In [65]:
tags_name = ["OrdType", "Side", "SecurityID"]

def exctract_block(record):
    try:
        body = record["body"]
        if body:
            new_obj = {
                "status": record.get("status"),
            }
            tags = Utils.search_fields(body["fields"], *tags_name)
            for tag_name in tags:
                tag = tags[tag_name]
                
                new_obj.update({
                    tag_name: tag[0].get("simpleValue"),
                })
            return new_obj
    except:
        return None

streams = set()
for record in events_tree.events.values():
    messages = record.get("attachedMessageIds")
    for msg in messages:
        streams.add(msg.split(":")[0])

messages: Data = data_source.get_messages_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    stream=list(streams),
)

In [64]:
compute = []
for record in messages.map(exctract_block):
    for tag_name in tags_name:
        new_obj = {
            "status": 'Unknown'
        }
        if isinstance(record.get(tag_name), list):
            for tag in record.get(tag_name):
                new_obj[tag_name] = tag
                compute.append(new_obj)
        else:
            new_obj.update({tag_name: record.get(tag_name)})
            compute.append(new_obj)

Utils.aggregate_several_group(compute)

,status,count
0,Unknown,279.0
1,Total,279.0
,OrdType,count
0,2,87.0
1,None,192.0
2,Total,279.0
,Side,count
0,1,51.0
1,2,36.0
2,None,192.0


## [4] Table - Types of messages and their number processed by recon 

In [26]:
def is_recon_ancestor(record):
    parent_id = record.get("parentEventId")
    if parent_id is not None:
        ancestor = events_tree.get_ancestor_by_super_type(record, "Recon Folder", get_super_type)
        if ancestor:
            return True
    return False

def is_match_or_match_failed(record):
    ancestor = events_tree.get_ancestor_by_name(record, "No match")
    if ancestor and not ancestor.get("successful"):
        return True
    ancestor = events_tree.get_ancestor_by_name(record, "Matched passed")
    if ancestor:
        return True
    return False

def exctract_block(record):
    messages_id = record.get("messages_id")
    
    if not messages_id:
        return None
        
    messages = data_source.find_messages_by_id_from_data_provider(messages_id)
    if isinstance(messages, dict):
        messages = [messages]
    output = [{"MsgType": message.get("body", {}).get("metadata", {}).get("messageType")} for message in messages]
    
    return output

events_with_attached_msgs = data_source.get_events_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    metadataOnly=False,
    attachedMessages=True,
)

data = events_with_attached_msgs\
        .map(extract_basic)\
        .filter(is_recon_ancestor)\
        .filter(is_match_or_match_failed)\
        .map(exctract_block)

df = DataFrame(data)
df
df.groupby("MsgType").size().reset_index(name="count")

,MsgType,count
0,ExecutionReport,18
1,NewOrderSingle,16
